<a href="https://colab.research.google.com/github/koyomin9zx/CS331.K11-Flowers-Recognition/blob/master/trainSoftmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip /content/drive/'My Drive'/ML/dataset.zip
!ln -s /content/drive/'My Drive'/ML/backup /content

In [0]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
from keras import models
from keras import layers
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.externals import joblib
import cv2

In [0]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [0]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
model = VGG16(weights='imagenet', include_top=False)

In [0]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [0]:
# Dùng pre-trained model để lấy ra các feature của ảnh
features = model.predict(list_image)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features = features.reshape((features.shape[0], 512*7*7))

In [0]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [0]:
model1 = models.Sequential()
model1.add(layers.Dense(5, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [85]:
history = model1.fit(X_train, y_train,epochs=100)

Epoch 1/100
3458/3458 [==============================] - 1s 147us/step - loss: 0.6848 - acc: 0.9560
Epoch 2/100
3458/3458 [==============================] - 0s 144us/step - loss: 0.6695 - acc: 0.9575
Epoch 3/100
3458/3458 [==============================] - 1s 147us/step - loss: 0.6867 - acc: 0.9566
Epoch 4/100
3458/3458 [==============================] - 1s 148us/step - loss: 0.6849 - acc: 0.9569
Epoch 5/100
3458/3458 [==============================] - 0s 141us/step - loss: 0.7891 - acc: 0.9488
Epoch 6/100
3458/3458 [==============================] - 0s 138us/step - loss: 0.9149 - acc: 0.9404
Epoch 7/100
3458/3458 [==============================] - 0s 141us/step - loss: 0.7211 - acc: 0.9517
Epoch 8/100
3458/3458 [==============================] - 1s 152us/step - loss: 0.6778 - acc: 0.9563
Epoch 9/100
3458/3458 [==============================] - 0s 142us/step - loss: 0.6628 - acc: 0.9558
Epoch 10/100
3458/3458 [==============================] - 0s 139us/step - loss: 0.6229 - acc: 0.9598

In [86]:
y_pred = model1.predict(X_test,verbose=0)
rounded_labels=np.argmax(y_test, axis=1)
rounded_pred=np.argmax(y_pred, axis=1)
print(confusion_matrix(rounded_labels,rounded_pred))
print(classification_report(rounded_labels,rounded_pred))

[[124  11   2   5   4]
 [  8 190   1   6   3]
 [  4   5 143   5  19]
 [  7   9   2 130   1]
 [  3   4  14  10 155]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       146
           1       0.87      0.91      0.89       208
           2       0.88      0.81      0.85       176
           3       0.83      0.87      0.85       149
           4       0.85      0.83      0.84       186

    accuracy                           0.86       865
   macro avg       0.86      0.86      0.86       865
weighted avg       0.86      0.86      0.86       865

